In [1]:
!unzip /content/captcha_dataset.zip


Archive:  /content/captcha_dataset.zip
  inflating: ml_test_3188.csv        
  inflating: ml_data_3188.csv        
 extracting: samples/bc8nf.png       
  inflating: samples/bnc2f.png       
 extracting: samples/c6we6.png       
 extracting: samples/c7gb3.png       
 extracting: samples/cdmn8.png       
 extracting: samples/d3ycn.png       
 extracting: samples/d66cn.png       
 extracting: samples/d7en3.png       
 extracting: samples/d8dce.png       
 extracting: samples/ddmyg.png       
 extracting: samples/defyx.png       
 extracting: samples/e43ym.png       
 extracting: samples/e667x.png       
 extracting: samples/e7nx4.png       
  inflating: samples/e8dxn.png       
 extracting: samples/ef4np.png       
  inflating: samples/eppg3.png       
  inflating: samples/f6bpw.jpg       
  inflating: samples/fcne6.png       
 extracting: samples/fg8n4.png       
 extracting: samples/g247w.png       
  inflating: samples/g7gnf.png       
 extracting: samples/gn2xy.png       
 extracting

In [16]:
import os
import string

chars = string.ascii_lowercase + string.digits
print(len(chars))

captchas = os.listdir("/content/samples/")
captchas = list(map(lambda x: x.split(".")[0], captchas))
print(n := len(captchas))

imgshape=(50,200,1)

character= string.ascii_lowercase + "0123456789"
nchar = len(character)

36
1070


In [3]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import os
from keras import layers
from keras.models import Model
from keras.models import load_model
from keras import callbacks
import cv2
import string

In [4]:

def preprocess():
  X = np.zeros((n,50,200,1))
  y = np.zeros((5,n,nchar))

  for i, pic in enumerate(os.listdir("/content/samples")):

    img = cv2.imread(os.path.join("/content/samples", pic), cv2.IMREAD_GRAYSCALE)
    pic_target = pic[:-4]

    if len(pic_target) < 6:
      img = img / 255.0
      img = np.reshape(img, (50, 200, 1))

      target=np.zeros((5,nchar))

      for j, k in enumerate(pic_target):
         index = character.find(k)
         target[j, index] = 1

      X[i] = img
      y[:,i] = target

  return X,y



In [5]:

def createmodel():
    img = layers.Input(shape=imgshape)
    conv1 = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(img)
    mp1 = layers.MaxPooling2D(padding='same')(conv1)
    conv2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(conv2)
    conv3 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp2)
    bn = layers.BatchNormalization()(conv3)
    mp3 = layers.MaxPooling2D(padding='same')(bn)

    flat = layers.Flatten()(mp3)

    outs = []
    for _ in range(5):
        dens1 = layers.Dense(64, activation='relu')(flat)
        drop = layers.Dropout(0.5)(dens1)
        res = layers.Dense(nchar, activation='sigmoid')(drop)

        outs.append(res)

    model = Model(img, outs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy","accuracy","accuracy","accuracy","accuracy"])
    return model


In [6]:
model = createmodel()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 50, 200, 1)     │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 50, 200, 16)    │            160 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 25, 100, 16)    │              0 │ conv2d[0][0]           │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 25, 100, 32)    │          4,640 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 13, 50, 32)     │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 13, 50, 32)     │          9,248 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 13, 50, 32)     │            128 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 7, 25, 32)      │              0 │ batch_normalization[0… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 5600)           │              0 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │        358,464 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 64)             │        358,464 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 64)             │        358,464 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 64)             │        358,464 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 64)             │        358,464 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 64)             │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 64)             │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 64)             │              0 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)  

 Total params: 1,818,196 (6.94 MB)

 Trainable params: 1,818,132 (6.94 MB)

 Non-trainable params: 64 (256.00 B)

In [7]:

X,y=preprocess()

In [8]:
from sklearn.model_selection import train_test_split

n_samples = X.shape[0]
y_reshaped = y.transpose(1, 0, 2).reshape(n_samples, -1)


X_train, X_test, y_train_reshaped, y_test_reshaped = train_test_split(
    X, y_reshaped, test_size=0.3, random_state=42
)


y_train = y_train_reshaped.reshape(-1, 5, y.shape[2]).transpose(1, 0, 2)
y_test = y_test_reshaped.reshape(-1, 5, y.shape[2]).transpose(1, 0, 2)


In [9]:
def get_test_data_back(y_test_encoded):
  y_test = []
  for i in range:



SyntaxError: incomplete input (<ipython-input-9-f2d004f50149>, line 4)

In [10]:
g = []
for i in range(321):
    gh = np.argmax(y_test[:, i], axis=1)  # Get indices of max values
    s = "".join(character[j] for j in gh)  # Convert indices to string
    g.append(s)
print(g)



['c4527', '67dey', '77n6g', 'yd755', '5n732', 'y7x8p', '75pfw', 'nbwpn', '66wp5', 'pmg55', '6mn8n', '264m5', '8d4wm', '84w7x', 'm23bp', '28348', '3p4nn', '8e32m', 'gm7n8', 'e8e5e', 'wyc25', 'w4x2m', 'n7meb', 'gecmf', 'pp546', 'b84xc', 'm67b3', 'mcg43', '445cc', '7gce6', 'nny5e', 'deep5', '573bn', 'xxbm5', 'xfgxb', 'bmxpe', 'ef4mn', '72m6f', '32cnn', '56m6y', 'e6b7y', 'n7ebx', '2mpnn', 'pgm2e', 'wg625', '2ycn8', 'xyyyw', 'pxdwp', 'x3deb', 'xcf88', 'pw5nc', 'dnne7', 'gfp54', 'n7ff2', '5bb66', 'nwfde', 'mmg38', 'e25xg', 'p8c24', '22d5n', 'p2ym2', '7fmcy', 'f75cx', 'b35f6', 'gn2d3', 'gm6nn', '428b6', '8gf7n', 'p2dw7', '88bgx', 'nf2n8', 'ygenn', 'f6bpw', 'x4f7g', 'f753f', 'fbp2c', 'cd4eg', 'nfd8g', 'gp7c5', '4yc85', 'x3fwf', '44c22', 'xnd3y', '8xef7', 'm4g8g', 'mmc5n', 'f6ne5', 'c353e', 'dc436', 'bdbb3', '8g4yp', 'bnc2f', 'defyx', 'xdcn4', '33b22', '6dd2y', 'y5w28', '56c34', 'd2ycw', 'pnnwy', 'e6m6p', 'ebcbx', 'wb3ed', '3n3cf', '74853', 'wm47f', '85622', 'dbfen', '44fyb', '68wfd', 'yf347', 

In [12]:
hist = model.fit(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]], batch_size=32, epochs=50, validation_split=0.2)

Epoch 1/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 345ms/step - dense_1_accuracy: 0.0468 - dense_1_loss: 3.7398 - dense_3_accuracy: 0.0503 - dense_3_loss: 3.8908 - dense_5_accuracy: 0.0400 - dense_5_loss: 3.6622 - dense_7_accuracy: 0.0274 - dense_7_loss: 3.6999 - dense_9_accuracy: 0.0410 - dense_9_loss: 3.6387 - loss: 18.6328 - val_dense_1_accuracy: 0.0600 - val_dense_1_loss: 3.5374 - val_dense_3_accuracy: 0.0533 - val_dense_3_loss: 3.5346 - val_dense_5_accuracy: 0.0533 - val_dense_5_loss: 3.5254 - val_dense_7_accuracy: 0.0667 - val_dense_7_loss: 3.5369 - val_dense_9_accuracy: 0.1000 - val_dense_9_loss: 3.5343 - val_loss: 17.6614
Epoch 2/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 307ms/step - dense_1_accuracy: 0.0819 - dense_1_loss: 3.3765 - dense_3_accuracy: 0.0549 - dense_3_loss: 3.4582 - dense_5_accuracy: 0.0517 - dense_5_loss: 3.3484 - dense_7_accuracy: 0.0627 - dense_7_loss: 3.3749 - dense_9_accuracy: 0.0612 - dense_9_loss: 3.4321 - loss: 16.9904 - val_dense_1_accuracy: 0.0933 - val_dense_1_loss: 3.

In [13]:

def predict(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

    if img is not None:
        img = img / 255.0
    else:
        print("Not detected");

    res = np.array(model.predict(img[np.newaxis, :, :, np.newaxis]))
    result = np.reshape(res, (5, 36))
    k_ind = []
    probs = []
    for i in result:
        k_ind.append(np.argmax(i))

    capt = ''
    for k in k_ind:
        capt += character[k]
    return capt

In [14]:
# img=cv2.imread('/content/thresh.png',cv2.IMREAD_GRAYSCALE)
# img = cv2.resize(img, (200, 50))
# cv2.imwrite("/content/thresh.png", img)

pred = predict("/content/samples/67dey.png")
print(pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
678ey


In [ ]:
results = model.evaluate(X_test, [y_test[i] for i in range(5)])

for i, metric in enumerate(model.metrics_names):
    print(f"{metric}: {results[i]}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - dense_1_accuracy: 0.9709 - dense_1_loss: 0.0736 - dense_3_accuracy: 0.9222 - dense_3_loss: 0.3741 - dense_5_accuracy: 0.8657 - dense_5_loss: 0.6027 - dense_7_accuracy: 0.7264 - dense_7_loss: 1.4542 - dense_9_accuracy: 0.8114 - dense_9_loss: 0.9499 - loss: 3.4949
loss: 3.4471640586853027
compile_metrics: 0.09829118102788925
dense_1_loss: 0.42162859439849854
dense_3_loss: 0.6670646667480469
dense_5_loss: 1.2467617988586426
dense_7_loss: 0.7707275748252869
dense_9_loss: 0.9657320976257324


In [ ]:
import numpy as np

y_pred = model.predict(X_test)
y_pred_indices = np.argmax(y_pred, axis=2)

y_test_indices = np.argmax(y_test, axis=2)

correct_predictions = (y_pred_indices == y_test_indices)

position_accuracies = np.mean(correct_predictions, axis=1)
overall_accuracy = np.mean(correct_predictions)

print("Position-wise Accuracies:", position_accuracies)
print("Overall Accuracy:", overall_accuracy)


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step
Position-wise Accuracies: [0.88785047 0.88785047 0.80373832 0.72274143 0.78504673]
Overall Accuracy: 0.8174454828660436
